In [20]:
import os
os.chdir('/opt/ml/code/mmdetection_trash')

In [19]:
!unzip skf1.zip

Archive:  skf1.zip
  inflating: Stratified KFold Json 파일/valid_data4.json  
  inflating: Stratified KFold Json 파일/valid_data1.json  
  inflating: Stratified KFold Json 파일/valid_data0.json  
  inflating: Stratified KFold Json 파일/valid_data2.json  
  inflating: Stratified KFold Json 파일/valid_data3.json  
  inflating: Stratified KFold Json 파일/train_data1.json  
  inflating: Stratified KFold Json 파일/train_data2.json  
  inflating: Stratified KFold Json 파일/train_data3.json  


In [6]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [73]:
# classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
#            "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# classes = ("UNKNOWN", "Paper pack", "Metal", "Glass", "Styrofoam", "Battery", "Clothing")
classes = ("UNKNOWN", "General trash", "Paper",
           "Plastic", "Plastic bag")
# config file 들고오기
cfg = Config.fromfile('./configs/cascade_rcnn/cascade_rcnn_x101_64x4d_fpn_20e_coco.py')

PREFIX = '../../input/data/'

epoch = 15

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.data.samples_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/cascade_rcnn_x101_64x4d_fpn_20e_coco_the_rest'

# cfg.model.roi_head.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, 'best_bbox_mAP_50.pth')

In [74]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [75]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

2021-05-19 22:05:15,266 - mmdet - INFO - load model from: open-mmlab://resnext101_64x4d
2021-05-19 22:05:15,267 - mmdet - INFO - Use load_from_openmmlab loader


Use load_from_local loader


In [76]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 7.9 task/s, elapsed: 106s, ETA:     0s

In [77]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 7
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, '2model_rare_final.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,1 0.6431669 168.69246 78.26431 251.68831 152.4...,batch_01_vt/0021.jpg
1,2 0.13937216 13.619583 247.24962 313.18402 460...,batch_01_vt/0028.jpg
2,1 0.721683 74.66969 295.38107 195.94081 379.12...,batch_01_vt/0031.jpg
3,2 0.152701 23.697998 220.01796 391.10822 375.1...,batch_01_vt/0032.jpg
4,1 0.082837395 245.84119 267.52896 384.56268 37...,batch_01_vt/0070.jpg
